In [30]:
from PIL import Image
from skimage.io import imread_collection , concatenate_images
from sklearn.neighbors import NearestNeighbors
from sklearn import preprocessing, svm
import numpy as np
import matplotlib.pyplot as plt
import pywt as ptw
import copy as cp
from skimage.io.manage_plugins import _inject_imread_collection_if_needed

In [31]:
def draw(image):
    plt.imshow(image)
    plt.show()
    return

def hwavelet(images, n):
    cA = images
    for i in range(n):
        cA, (cH, cV, cD) = ptw.dwt2(cA, 'haar')
    return cA

def normalizar(dataset):
    aux = []
    for i in range(len(dataset)):
        aux = aux + [dataset[i].reshape(24*24).tolist()]
    return aux

In [32]:
ANGER = 0
CONTEMPT = 1
DISGUST = 2
FEAR = 3
HAPPY = 4
SADNESS = 5
SURPRISE = 6

# paths
pathAnger = 'images/anger/*.png'
pathContempt = 'images/contempt/*.png'
pathDisgust = 'images/disgust/*.png'
pathFear = 'images/fear/*.png'
pathHappy = 'images/happy/*.png'
pathSadness = 'images/sadness/*.png'
pathSurprise = 'images/surprise/*.png'

#images
imagesAnger = imread_collection(pathAnger)
imagesContempt = imread_collection(pathContempt)
imagesDisgust = imread_collection(pathDisgust)
imagesFear = imread_collection(pathFear)
imagesHappy = imread_collection(pathHappy)
imagesSadness = imread_collection(pathSadness)
imagesSurprise = imread_collection(pathSurprise)

# Haar Wavelet
imagesAngerHW = hwavelet(imagesAnger, 1)
imagesContemptHW = hwavelet(imagesContempt, 1)
imagesDisgustHW = hwavelet(imagesDisgust, 1)
imagesFearHW = hwavelet(imagesFear, 1)
imagesHappyHW = hwavelet(imagesHappy, 1)
imagesSadnessHW = hwavelet(imagesSadness, 1)
imagesSurpriseHW = hwavelet(imagesSurprise, 1)

# Normalizacion
anger = normalizar(imagesAngerHW)
contempt = normalizar(imagesContemptHW)
disgust = normalizar(imagesDisgustHW)
fear = normalizar(imagesFearHW)
happy = normalizar(imagesHappyHW)
sadness = normalizar(imagesSadnessHW)
surprise = normalizar(imagesSurpriseHW)

def target_vector():
  return ['ANGER']*len(anger) + ['CONTEMPT']*len(contempt) + ['DISGUST']*len(disgust) \
    + ['FEAR']*len(fear) + ['HAPPY']*len(happy) + ['SADNESS']*len(sadness) \
    + ['SURPRISE']*len(surprise)

X = anger + contempt + disgust + fear + happy + sadness + surprise
Y = target_vector()

print(len(X))
print(len(Y))

ajuste = svm.SVC(gamma=0.001, C=100)
ajuste.fit(X, Y)

molesto = 'test/S010_004_00000017.png'

img = Image.open(molesto)
img_wavelet = hwavelet(img, 1)
normalized_img = normalizar([img_wavelet])[0]
# print(len(normalized_img))
print('prediction: ', ajuste.predict([normalized_img]))
    

978
978
prediction:  ['SURPRISE']
